In [65]:
import pandas as pd
# импортируем библиотеку для геокодирования 
from geopy.geocoders import Nominatim
import category_encoders as ce
import numpy as np # linear algebra


In [4]:
data = pd.read_csv('data/hotels.csv')
data.head()

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097
2,151 bis Rue de Rennes 6th arr 75006 Paris France,32,10/18/2016,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,14,7.5,"[' Leisure trip ', ' Solo traveler ', ' Modern...",289 day,48.845377,2.325643
3,216 Avenue Jean Jaures 19th arr 75019 Paris Fr...,34,9/22/2015,7.5,Mercure Paris 19 Philharmonie La Villette,United Kingdom,No Negative,0,607,Friendly staff quiet comfortable room spotles...,11,8,10.0,"[' Leisure trip ', ' Solo traveler ', ' Standa...",681 day,48.888697,2.394540
4,Molenwerf 1 1014 AG Amsterdam Netherlands,914,3/5/2016,8.5,Golden Tulip Amsterdam West,Poland,Torn sheets,4,7586,The staff was very friendly and helpful Break...,20,10,9.6,"[' Business trip ', ' Couple ', ' Standard Dou...",516 day,52.385601,4.847060


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               386803 non-null  object 
 1   additional_number_of_scoring                386803 non-null  int64  
 2   review_date                                 386803 non-null  object 
 3   average_score                               386803 non-null  float64
 4   hotel_name                                  386803 non-null  object 
 5   reviewer_nationality                        386803 non-null  object 
 6   negative_review                             386803 non-null  object 
 7   review_total_negative_word_counts           386803 non-null  int64  
 8   total_number_of_reviews                     386803 non-null  int64  
 9   positive_review                             386803 non-null  object 
 

In [6]:
X = data.drop(['reviewer_score'], axis = 1)  
y = data['reviewer_score'] 

In [7]:
y=y.astype('int')

In [8]:
data['hotel_name'].unique().shape[0]

1492

In [9]:
data['review_date'] = pd.to_datetime(data['review_date'])

In [10]:
data['review_date'].describe()

count                           386803
mean     2016-08-13 14:41:18.712936704
min                2015-08-04 00:00:00
25%                2016-02-23 00:00:00
50%                2016-08-15 00:00:00
75%                2017-02-09 00:00:00
max                2017-08-03 00:00:00
Name: review_date, dtype: object

In [11]:
# Разделение строк на списки и развертывание в отдельные строки
hotels_tags = data.copy()
hotels_tags['tags'] = hotels_tags['tags'].str.strip("[]").str.split("', '")
hotels_tags = hotels_tags.explode('tags')

# Удаление лишних пробелов и кавычек
hotels_tags['tags'] = hotels_tags['tags'].str.strip("' ")

# Получение уникальных значений
unique_values = hotels_tags['tags'].unique()

In [12]:
unique_values.shape[0]

2368

In [13]:
import ast
# Преобразуем строку тэгов в список
data['tags_str'] = data['tags']
data['tags'] = data['tags'].apply(ast.literal_eval)

In [19]:
data[data['tag_len']==6]['tags']

698       [ With a pet ,  Leisure trip ,  Solo traveler ...
1276      [ With a pet ,  Leisure trip ,  Group ,  Super...
1407      [ With a pet ,  Leisure trip ,  Family with yo...
2102      [ With a pet ,  Business trip ,  Couple ,  Kin...
2281      [ With a pet ,  Leisure trip ,  Group ,  2 roo...
                                ...                        
385187    [ With a pet ,  Business trip ,  Couple ,  Cla...
385499    [ With a pet ,  Leisure trip ,  Family with ol...
385641    [ With a pet ,  Leisure trip ,  Couple ,  Delu...
386348    [ With a pet ,  Leisure trip ,  Couple ,  Supe...
386645    [ With a pet ,  Leisure trip ,  Group ,  Super...
Name: tags, Length: 563, dtype: object

In [22]:
data['tags'].iloc[1407]

[' With a pet ',
 ' Leisure trip ',
 ' Family with young children ',
 ' Family Room ',
 ' Stayed 3 nights ',
 ' Submitted from a mobile device ']

In [ ]:
mask = data['tag_len'] == 5
data[mask]['tags'].head(30)

0     [ Leisure trip ,  Couple ,  Studio Suite ,  St...
2     [ Leisure trip ,  Solo traveler ,  Modern Doub...
5     [ Business trip ,  Solo traveler ,  Superior T...
6     [ Leisure trip ,  Couple ,  Double Room Non Sm...
9     [ Leisure trip ,  Couple ,  Twin Hilton Guest ...
13    [ Leisure trip ,  Couple ,  Double or Twin Roo...
14    [ Leisure trip ,  Couple ,  Double or Twin Roo...
16    [ Business trip ,  Solo traveler ,  Standard R...
17    [ Leisure trip ,  Group ,  Standard Room ,  St...
20    [ Leisure trip ,  Couple ,  Standard ,  Stayed...
22    [ Leisure trip ,  Couple ,  Twin Room Non Smok...
23    [ Leisure trip ,  Family with young children ,...
26    [ Leisure trip ,  Family with young children ,...
27    [ Leisure trip ,  Group ,  2 rooms ,  Stayed 2...
28    [ Leisure trip ,  Family with young children ,...
29    [ Leisure trip ,  Solo traveler ,  Double Stud...
30    [ Leisure trip ,  Couple ,  Double Room ,  Sta...
33    [ Business trip ,  Couple ,  Standard Doub

In [ ]:
data['business_trip'] = data['tags'].apply(lambda tags: 1 if any('business' in tag.lower() for tag in tags) else 0)

In [ ]:
data['mobile_device'] = data['tags'].apply(lambda tags: 1 if any('mobile device' in tag.lower() for tag in tags) else 0)

In [ ]:
def set_why_stayed(value):
  if any('group' in item.lower() for item in value):
    return 1
  if any('couple' in item for item in value):
    return 2  
  if any('solo' in item for item in value):
    return 3
  if any('young' in item for item in value):
    return 4
  if any('older' in item for item in value):
    return 5 
  return 0

data['who_stayed'] = data['tags'].apply(set_why_stayed)

In [ ]:
def set_room_type(value):
  if any('single room' in item.lower() for item in value): return 1
  if any('double room' in item.lower() for item in value): return 2
  if any('2 rooms' in item.lower() for item in value): return 3
  if any('standard double room' in item.lower() for item in value): return 4
  if any('standard double or twin room' in item.lower() for item in value): return 5
  if any('classic double room' in item.lower() for item in value): return 6
  if any('double or twin room' in item.lower() for item in value): return 7
  if any('superior double room' in item.lower() for item in value): return 8
  if any('superior double or twin room' in item.lower() for item in value): return 9
  if any('deluxe double room' in item.lower() for item in value): return 10
  if any('standard twin room' in item.lower() for item in value): return 11
  if any('twin room' in item.lower() for item in value): return 12
  return 0

  
data['tag_room_type'] = data['tags'].apply(set_room_type)

In [16]:
data['tag_len'] = data['tags'].apply(len)

In [23]:
data['with_pet'] = data['tags'].apply(lambda tags: 1 if any('a pet' in tag.lower() for tag in tags) else 0)

In [24]:
data['nights'] = data['tags_str'].str.extract(r"Stayed (\d+) night")
data['nights'] = data['nights'].fillna(data['nights'].dropna().astype(int).median()).astype(int)


In [25]:
# Выделяем такие признаки, как год, месяц и день недели
data['review_year'] = pd.to_datetime(data['review_date']).dt.year
data['review_month'] = pd.to_datetime(data['review_date']).dt.month
data['review_day_of_week'] = pd.to_datetime(data['review_date']).dt.day_of_week


In [26]:
# Подсчет частоты слов в описаниях
all_descriptions = ' '.join(data['negative_review'])
word_counts = pd.Series(all_descriptions.lower().split()).value_counts()
print(word_counts.head(30))

the          398224
was          177329
a            172606
to           171723
and          164247
no           148337
room         132025
in           126328
negative      97092
not           94357
i             91833
of            90443
for           88112
it            80405
we            74441
is            61689
very          60354
but           56659
hotel         55916
t             54108
on            51551
were          46215
at            45885
had           45194
that          44238
breakfast     43528
have          41060
with          38968
small         37372
there         34728
Name: count, dtype: int64


In [86]:
data['negative_review'] = data['negative_review'].str.lower()
data['positive_review'] = data['positive_review'].str.lower()

In [ ]:
data['negative_review'].unique()
data = data.drop('tag_type_trip_0', axis=1)
data['negative_review'].describe()
data['negative_review'].value_counts()
data.copy()

In [89]:
tags_list = ast.literal_eval("[' Leisure trip ', ' Couple ', ' Double Room ', ' Stayed 2 nights ']")
tags_list

[' Leisure trip ', ' Couple ', ' Double Room ', ' Stayed 2 nights ']

In [87]:
type(data['negative_review'])

pandas.core.series.Series

In [28]:
# Подсчет частоты слов в описаниях
all_descriptions = ' '.join(data['positive_review'])
word_counts = pd.Series(all_descriptions.lower().split()).value_counts()
print(word_counts.head(30))

the            385786
and            315806
was            177066
staff          145874
very           144568
location       144568
to             140847
a              123588
room           105728
hotel           94000
in              85671
good            84480
of              79844
great           79160
is              76765
for             68506
were            68308
friendly        63870
breakfast       63307
helpful         57201
nice            52027
we              51818
clean           50250
with            49532
excellent       46761
i               45776
comfortable     44843
it              43258
bed             37464
no              33827
Name: count, dtype: int64


In [29]:
positive_reviews_with_no = data[data['positive_review'].str.contains(' t positive')]

In [30]:
positive_reviews_with_no['positive_review'].unique()

array([], dtype=object)

In [31]:
def set_negative_flag(value):
  if 'no comment' in value or 'no negative' in value or 'nothing negativee' in value or \
  'not negative' in value or 'do not make it negative' in value or \
  'no comment' in value:
    return 0
  return 1

data['negative_flag'] = data['negative_review'].apply(set_negative_flag)

In [77]:
# Создаем новый признак, в котором отражаем, евляется ли отзыв негативным
def delete_word(value):
    value.replace('no comment','')
    value.replace('no negative','')
    value.replace('nothing negativee','')
    value.replace('not negative','')
    value.replace('do not make it negative','')
    value.replace('no comment','')
    return value

data['negative_review'] = data['negative_review'].apply(delete_word)

In [78]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Создаем экземпляр анализатора
sia = SentimentIntensityAnalyzer()

# Оцениваем текст отзывов и записываем результат в новый признак
data['negative_review_score'] = data['negative_review'].apply(sia.polarity_scores)

In [79]:
# Скоринг позитивного отзыва
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Создаем экземпляр анализатора
sia = SentimentIntensityAnalyzer()

# Оцениваем текст отзывов и записываем результат в новый признак
data['positive_review_score'] = data['positive_review'].apply(sia.polarity_scores)

In [84]:
col = data.columns
col

Index(['hotel_address', 'additional_number_of_scoring', 'review_date',
       'average_score', 'hotel_name', 'reviewer_nationality',
       'negative_review', 'review_total_negative_word_counts',
       'total_number_of_reviews', 'positive_review',
       'review_total_positive_word_counts',
       'total_number_of_reviews_reviewer_has_given', 'reviewer_score', 'tags',
       'days_since_review', 'lat', 'lng', 'tags_str', 'tag_len', 'with_pet',
       'nights', 'review_year', 'review_month', 'review_day_of_week',
       'negative_flag', 'negative_review_room', 'negative_review_very',
       'negative_review_breakfast', 'negative_review_small',
       'negative_review_staff', 'negative_review_bed',
       'negative_review_bathroom', 'positive_review_staff',
       'positive_review_very', 'positive_review_location',
       'positive_review_room', 'positive_review_good', 'positive_review_great',
       'positive_review_friendly', 'positive_review_breakfast',
       'positive_review_helpfu

In [85]:
data['tag_with_pet']

KeyError: 'tag_with_pet'

In [82]:
# отношение негативного к позитивному
data['ratio_review_score'] = data.apply(lambda row: (row['negative_review_score']['neg'] +1) / (row['positive_review_score']['pos']+1), axis=1)

In [32]:
data['negative_review_room'] = data['negative_review'].str.contains(' room').astype(int)
data['negative_review_very'] = data['negative_review'].str.contains(' very').astype(int)
data['negative_review_breakfast'] = data['negative_review'].str.contains('breakfast').astype(int)
data['negative_review_small'] = data['negative_review'].str.contains('small').astype(int)
data['negative_review_staff'] = data['negative_review'].str.contains('staff').astype(int)
data['negative_review_bed'] = data['negative_review'].str.contains(' bed').astype(int)
data['negative_review_bathroom'] = data['negative_review'].str.contains(' bathroom').astype(int)

In [33]:
data['positive_review_staff'] = data['positive_review'].str.contains('staff ').astype(int)
data['positive_review_very'] = data['positive_review'].str.contains('very ').astype(int)
data['positive_review_location'] = data['positive_review'].str.contains(' location').astype(int)
data['positive_review_room'] = data['positive_review'].str.contains(' room').astype(int)
data['positive_review_good'] = data['positive_review'].str.contains('good ').astype(int)
data['positive_review_great'] = data['positive_review'].str.contains('great ').astype(int)
data['positive_review_friendly'] = data['positive_review'].str.contains('friendly').astype(int)
data['positive_review_breakfast'] = data['positive_review'].str.contains('breakfast').astype(int)
data['positive_review_helpful'] = data['positive_review'].str.contains('helpful').astype(int)
data['positive_review_clean'] = data['positive_review'].str.contains('clean').astype(int)
data['positive_review_excellent'] = data['positive_review'].str.contains('excellent').astype(int)
data['positive_review_comfortable'] = data['positive_review'].str.contains('comfortable').astype(int)
data['positive_review_bed'] = data['positive_review'].str.contains(' bed ').astype(int)
data['positive_review_lovely'] = data['positive_review'].str.contains('lovely').astype(int)

In [34]:
rows_with_missing_lat = data[data['lat'].isnull() | (data['lat'] == '')]
rows_with_missing_lat = rows_with_missing_lat[['hotel_name', 'lat', 'lng', 'hotel_address']]
rows_with_missing_lat = rows_with_missing_lat.drop_duplicates(subset='hotel_address')

In [35]:
geolocator = Nominatim(user_agent="for_kaggle_test")

for i, row in rows_with_missing_lat.iterrows():
  hotel_address = row['hotel_address']
  location = geolocator.geocode(hotel_address, timeout=None)
  if not location:
    index = ' '.join(hotel_address.split()[-3:])
    location = geolocator.geocode(index, timeout=None)   
  if location:
    rows_with_missing_lat.loc[i, 'lat'] = location.latitude
    rows_with_missing_lat.loc[i, 'lng'] = location.longitude



In [36]:
for i, address in enumerate(data['hotel_address']):
    if pd.isna(data.at[i, 'lat']) or data.at[i, 'lat'] == '':
        matching_address = rows_with_missing_lat.loc[rows_with_missing_lat['hotel_address'] == address]
        if not matching_address.empty:
            data.at[i, 'lat'] = matching_address['lat'].values[0]
            data.at[i, 'lng'] = matching_address['lng'].values[0]

print(data)

                                            hotel_address  \
0       Stratton Street Mayfair Westminster Borough Lo...   
1       130 134 Southampton Row Camden London WC1B 5AF...   
2        151 bis Rue de Rennes 6th arr 75006 Paris France   
3       216 Avenue Jean Jaures 19th arr 75019 Paris Fr...   
4               Molenwerf 1 1014 AG Amsterdam Netherlands   
...                                                   ...   
386798  9 Knaresborough Place Kensington and Chelsea L...   
386799  Landstra er Hauptstra e 155 03 Landstra e 1030...   
386800  29 31 Gower Street Camden London WC1E 6HG Unit...   
386801  31 Great Cumberland Place Westminster Borough ...   
386802  25 Courtfield Gardens Kensington and Chelsea L...   

        additional_number_of_scoring review_date  average_score  \
0                                581  2016-02-19            8.4   
1                                299  2017-01-12            8.3   
2                                 32  2016-10-18            8.9   

In [37]:
hotels = data[['hotel_name', 'lat', 'lng', 'hotel_address']]
hotels = hotels.drop_duplicates(subset='hotel_name')

In [38]:
min_review_dates = data.groupby('hotel_name')['review_date'].min()
max_review_dates = data.groupby('hotel_name')['review_date'].max()

In [ ]:
min_review_dates


hotel_name
11 Cadogan Gardens                   2015-08-30
1K Hotel                             2015-08-06
25hours Hotel beim MuseumsQuartier   2015-08-04
41                                   2015-08-11
45 Park Lane Dorchester Collection   2015-09-10
                                        ...    
citizenM London Bankside             2015-08-04
citizenM London Shoreditch           2016-09-04
citizenM Tower of London             2016-07-14
every hotel Piccadilly               2015-08-07
pentahotel Vienna                    2015-08-18
Name: review_date, Length: 1492, dtype: datetime64[ns]

In [39]:

data['min_review_dates'] = data['hotel_name'].map(min_review_dates)
data['max_review_dates'] = data['hotel_name'].map(max_review_dates)

In [40]:
data['days_from_first_review_date'] = (data['review_date'] - data['min_review_dates']).dt.days

In [41]:
data['days_from_max_min_review_date'] = (data['max_review_dates'] - data['min_review_dates']).dt.days

In [ ]:
# Создание экземпляра OneHotEncoder
encoder = ce.OneHotEncoder()

# Преобразование признака 'reviewer_nationality' в матрицу one-hot
X = encoder.fit_transform(data['reviewer_nationality'])

# Объединение закодированных столбцов с исходным DataFrame
data = pd.concat([data, X], axis=1)

print(data)

                                            hotel_address  \
0       Stratton Street Mayfair Westminster Borough Lo...   
1       130 134 Southampton Row Camden London WC1B 5AF...   
2        151 bis Rue de Rennes 6th arr 75006 Paris France   
3       216 Avenue Jean Jaures 19th arr 75019 Paris Fr...   
4               Molenwerf 1 1014 AG Amsterdam Netherlands   
...                                                   ...   
386798  9 Knaresborough Place Kensington and Chelsea L...   
386799  Landstra er Hauptstra e 155 03 Landstra e 1030...   
386800  29 31 Gower Street Camden London WC1E 6HG Unit...   
386801  31 Great Cumberland Place Westminster Borough ...   
386802  25 Courtfield Gardens Kensington and Chelsea L...   

        additional_number_of_scoring review_date  average_score  \
0                                581  2016-02-19            8.4   
1                                299  2017-01-12            8.3   
2                                 32  2016-10-18            8.9   

In [ ]:
data

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,...,reviewer_nationality_216,reviewer_nationality_217,reviewer_nationality_218,reviewer_nationality_219,reviewer_nationality_220,reviewer_nationality_221,reviewer_nationality_222,reviewer_nationality_223,reviewer_nationality_224,reviewer_nationality_225
0,Stratton Street Mayfair Westminster Borough Lo...,581,2016-02-19,8.4,The May Fair Hotel,United Kingdom,leaving,3,1994,staff were amazing,...,0,0,0,0,0,0,0,0,0,0
1,130 134 Southampton Row Camden London WC1B 5AF...,299,2017-01-12,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,...,0,0,0,0,0,0,0,0,0,0
2,151 bis Rue de Rennes 6th arr 75006 Paris France,32,2016-10-18,8.9,Legend Saint Germain by Elegancia,China,no kettle in room,6,406,no positive,...,0,0,0,0,0,0,0,0,0,0
3,216 Avenue Jean Jaures 19th arr 75019 Paris Fr...,34,2015-09-22,7.5,Mercure Paris 19 Philharmonie La Villette,United Kingdom,no negative,0,607,friendly staff quiet comfortable room spotles...,...,0,0,0,0,0,0,0,0,0,0
4,Molenwerf 1 1014 AG Amsterdam Netherlands,914,2016-03-05,8.5,Golden Tulip Amsterdam West,Poland,torn sheets,4,7586,the staff was very friendly and helpful break...,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386798,9 Knaresborough Place Kensington and Chelsea L...,107,2017-04-19,9.0,Hotel Moonlight,France,no negative,0,617,tr s proche du metro earl s court,...,0,0,0,0,0,0,0,0,0,0
386799,Landstra er Hauptstra e 155 03 Landstra e 1030...,272,2017-02-13,8.4,BEST WESTERN PLUS Amedia Wien,Turkey,no negative,0,3224,the bed was so comfy i stayed with my boyfrie...,...,0,0,0,0,0,0,0,0,0,0
386800,29 31 Gower Street Camden London WC1E 6HG Unit...,457,2016-02-07,6.8,Bloomsbury Palace Hotel,Netherlands,room is really small but guess is normal in l...,12,2751,great location simple check in out nice shower,...,0,0,0,0,0,0,0,0,0,0
386801,31 Great Cumberland Place Westminster Borough ...,365,2017-05-21,8.1,The Marble Arch London,United Arab Emirates,no negative,0,1567,location and very comfy bed,...,0,0,0,0,0,0,0,0,0,0


In [42]:
# Создание нового признака 'country' и 'city'
data['country'] = data['hotel_address'].apply(lambda x: x.split()[-1] 
        if x.split()[-1] != 'Kingdom' 
        else ' '.join(x.split()[-2:]))

data['city'] = data.apply(lambda x: x['hotel_address'].split()[-5] 
        if x['country'] == 'United Kingdom'
        else x['hotel_address'].split()[-2], axis=1
)

In [43]:
display(data[['country', 'city']].value_counts())

country         city     
United Kingdom  London       196774
Spain           Barcelona     45132
France          Paris         44830
Netherlands     Amsterdam     43006
Austria         Vienna        29178
Italy           Milan         27883
Name: count, dtype: int64

In [44]:
data['country'].unique()

array(['United Kingdom', 'France', 'Netherlands', 'Italy', 'Austria',
       'Spain'], dtype=object)

In [45]:
city_info = pd.DataFrame(
    [
        ['London', 10], 
        ['Barcelona', 9], 
        ['Paris', 1], 
        ['Amsterdam', 5], 
        ['Vienna', 18], 
        ['Milan', 13]
    ],     columns=['city', 'city_​​attractiveness_rating'])



In [46]:
data = data.merge(city_info, how='left', on='city')

In [47]:
encoder = ce.OneHotEncoder(cols=['city']) # указываем столбец для кодирования
encoder = encoder.fit_transform(data['city'])
X = encoder.rename(columns={'city_1':'London','city_2':'Paris','city_3':'Amsterdam','city_4':'Milan','city_5':'Vienna','city_6':'Barcelona'})
hotels = pd.concat([data, X], axis=1)

In [48]:
# Удаление пробелов из 'reviewer_nationality'
data['reviewer_nationality'] = data['reviewer_nationality'].str.strip()
# Создание нового признака 'local_resident'
data['local_resident'] = data['reviewer_nationality'] == data['country']

# Преобразование 'local_resident' в int
data['local_resident'] = data['local_resident'].astype('int')

In [49]:
data['local_resident'].unique()

array([1, 0])

In [50]:
data['reviewer_nationality']

0               United Kingdom
1               United Kingdom
2                        China
3               United Kingdom
4                       Poland
                  ...         
386798                  France
386799                  Turkey
386800             Netherlands
386801    United Arab Emirates
386802               Australia
Name: reviewer_nationality, Length: 386803, dtype: object

In [51]:
data['country']

0         United Kingdom
1         United Kingdom
2                 France
3                 France
4            Netherlands
               ...      
386798    United Kingdom
386799           Austria
386800    United Kingdom
386801    United Kingdom
386802    United Kingdom
Name: country, Length: 386803, dtype: object

In [52]:

# Создание нового признака 'review_pos_count'
data['review_pos_count'] = data.loc[data['negative_flag'] == 0].groupby('hotel_name')['review_date'].rank(method='dense', ascending=True).astype(int)
# Создание нового признака 'review_neg_count'
data['review_neg_count'] = data.loc[data['negative_flag'] == 1].groupby('hotel_name')['review_date'].rank(method='dense', ascending=True).astype(int)




In [53]:
data['review_count'] = data.groupby('hotel_name')['review_date'].rank(method='dense', ascending=True).astype(int)

In [54]:
data['review_count'].isnull().sum()

0

In [55]:
data['review_pos_count'].isnull().sum()

290493

In [61]:
df_city = data.drop_duplicates(subset='city')

In [62]:
df_city = df_city[['city']]

In [67]:
df_city.loc[:, 'city_lng'] = np.nan
df_city.loc[:, 'city_lat'] = np.nan

In [68]:
# Инструмент для поиска данных OSM по имени и адресу
geolocator = Nominatim(user_agent="for_kaggle_test")

# Заполняем отсутствующие координаты
for i, row in df_city.iterrows():
    city_address = row['city']
    location = geolocator.geocode(city_address, timeout=None)
    # Если по адресу не нашли, то ищем по индексу
    if not location:
        print(city_address) 
    if location:
        df_city.loc[i, 'city_lat'] = location.latitude
        df_city.loc[i, 'city_lng'] = location.longitude

In [75]:
from geopy.distance import geodesic

# Создание словаря для быстрого поиска координат городов
city_coords = dict(zip(df_city['city'], zip(df_city['city_lat'], df_city['city_lng'])))

# Вычислить расстояние между точками и записать в новый признак
data['distance_from_center'] = data.apply(lambda row: geodesic(city_coords[row['city']], (row['lat'], row['lng'])).km, axis=1)


In [76]:
data['distance_from_center']

0         1.105422
1         1.543394
2         1.557880
3         6.391716
4         3.390634
            ...   
386798    4.641050
386799    2.677258
386800    1.502941
386801    2.399640
386802    4.620117
Name: distance_from_center, Length: 386803, dtype: float64

In [74]:
from geopy.distance import geodesic
data.loc[:, 'distance_from_center'] = np.nan
distance = float('inf')

for i, city in enumerate(data['city']):
    city_row = df_city[df_city['city'] == city]
    city_lat = city_row['city_lat'].values[0]
    city_lng = city_row['city_lng'].values[0]
    data_lat = data.at[i, 'lat']
    data_lng = data.at[i, 'lng']
 
    data.at[i, 'distance_from_center'] = geodesic((city_lat, city_lng), (data_lat, data_lng)).km
    #print(city, city_lat, city_lng, data_lat, data_lng, data.at[i, 'distance_from_center'])


London 51.5074456 -0.1277653 51.5078943 -0.1436707 1.1054216055390684
London 51.5074456 -0.1277653 51.5210085 -0.1230966 1.543394490330983
Paris 48.8588897 2.3200410217200766 48.8453774 2.3256427 1.5578798185521652
Paris 48.8588897 2.3200410217200766 48.8886969 2.3945403 6.39171604889752
Amsterdam 52.3730796 4.8924534 52.385601 4.8470597 3.3906342200160906
Milan 45.4641943 9.1896346 45.4481878 9.2570459 5.564834794834984
London 51.5074456 -0.1277653 51.4942305 -0.1851141 4.245009491195826
Vienna 48.2083537 16.3725042 48.2085627 16.3541687 1.3629699319620887
London 51.5074456 -0.1277653 51.5001979 -0.1795357 3.6839953375552748
London 51.5074456 -0.1277653 51.5051084 -0.213266 5.942089176861886
Amsterdam 52.3730796 4.8924534 52.385601 4.8470597 3.3906342200160906
London 51.5074456 -0.1277653 51.4920287 -0.1917801 4.7646929158641225
London 51.5074456 -0.1277653 51.4930905 -0.1755022 3.6795448918372817
London 51.5074456 -0.1277653 51.4942305 -0.1851141 4.245009491195826
Barcelona 41.382893

KeyboardInterrupt: 

In [ ]:
import requests
from bs4 import BeautifulSoup

query = "хилтон"
url = f"https://www.google.com/search?q={query}"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

result_stats = soup.find("div", {"id": "result-stats"})
if result_stats:
    result_count = int(result_stats.text.split(" ")[1].replace(",", ""))
    print(f"Найдено результатов: {result_count}")
else:
    print("Не удалось получить количество найденных результатов")

Не удалось получить количество найденных результатов


In [ ]:

url = 'http://yandex.ru/yandsearch?text=%(q)s'
payload = {'q': 'Python',}
r = requests.get(url % payload)
soup = BeautifulSoup(r.text)
titles = [item.text for item in soup.findAll('div', attrs={'class': 'b-serp-item__text'})]

In [ ]:
r.text

'<!doctype html><html prefix="og: http://ogp.me/ns#" lang="ru"><meta http-equiv="X-UA-Compatible" content="IE=edge"><meta charset="utf-8"><meta name="viewport" content="width=device-width,initial-scale=1">\n    <link rel="icon" href="https://yastatic.net/s3/home-static/_/a6/a6a296b741b51880ae8a9b04a67cfe3f.png" sizes="16x16">\n    <link rel="icon" href="https://yastatic.net/s3/home-static/_/f4/f47b1b3d8194c36ce660324ab55a04fe.png" sizes="32x32">\n    <link rel="icon" href="https://yastatic.net/s3/home-static/_/f0/f0597b6727cc67dceebc4e3a87caf571.png" sizes="192x192">\n    <link rel="apple-touch-icon" href="https://yastatic.net/s3/home-static/_/a7/a79b81aa025e9edb2244e38581c868ad.png" sizes="152x152">\n    <link rel="apple-touch-icon" href="https://yastatic.net/s3/home-static/_/46/462e92b9e3792be37a1c3fdefb26af28.png" sizes="180x180">\n<title data-react-helmet="true">Ð\x9eÐ¹, Ð\x9aÐ°Ð¿Ñ\x87Ð°!</title><meta data-react-helmet="true" property="og:title" content="Ð¯Ð½Ð´ÐµÐºÑ\x81"><meta data

In [ ]:
api_url="http://127.0.0.1:5000/data"

def get_number_of_jobs_T(technology):
    number_of_jobs = 0
    payload={"Key Skills":technology}
    r=requests.get(api_url,params=payload)
    data = r.json()
    number_of_jobs += len(data)
    return technology,number_of_jobs

In [ ]:
hotels['category_reviewer_nationality'] = hotels['reviewer_nationality'].astype('category')
hotels['category_reviewer_nationality'] = hotels['category_reviewer_nationality'].cat.codes

In [ ]:
query = "python"
url = f"https://www.google.com/search?q={query}"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

result_stats = soup.find("div", {"id": "result-stats"})
if result_stats:
    result_count = int(result_stats.text.split(" ")[1].replace(",", ""))
    print(f"Found {result_count} results")
else:
    print("Unable to retrieve the number of search results")

Unable to retrieve the number of search results


In [ ]:
lat_mode = hotels['lat'].mode().iloc[0]

# Заполнение пустых значений в 'lat' модой
hotels['lat'] = hotels['lat'].fillna(lat_mode)

In [ ]:
# Получение моды для 'lng'
lng_mode = hotels['lng'].mode().iloc[0]

# Заполнение пустых значений в 'lng' модой
hotels['lng'] = hotels['lng'].fillna(lng_mode)

In [ ]:
# убираем признаки которые еще не успели обработать, 
# модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
object_columns = [s for s in hotels.columns if hotels[s].dtypes == 'object']
hotels.drop(object_columns, axis = 1, inplace=True)
hotels.drop('review_date', axis = 1, inplace=True)

In [ ]:
hotels.info()

In [ ]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х - данные с информацией об отелях, у - целевая переменная (рейтинги отелей)  
X = hotels.drop(['reviewer_score'], axis = 1)  
y = hotels['reviewer_score'] 

In [ ]:
y=y.astype('int')
from sklearn.feature_selection import chi2 # хи-квадрат

In [ ]:
# непрерывные признаки
num_cols = ['total_number_of_reviews_reviewer_has_given', 'additional_number_of_scoring', 'review_total_negative_word_counts', 'total_number_of_reviews', 'review_total_positive_word_counts' ]

# категориальные признаки
cat_cols = ['average_score', 'lat', 'reviewer_score', 'lng', 'negative_flag', 'positive_flag', 'review_year', 'review_month', 'review_day_of_week', 'nights']

In [ ]:
# непрерывные признаки
num_cols = ['total_number_of_reviews_reviewer_has_given', 'additional_number_of_scoring', 'review_total_negative_word_counts', 'total_number_of_reviews', 'review_total_positive_word_counts' ]

# категориальные признаки
cat_cols = ['average_score', 'lat', 'negative_flag', 'positive_flag', 'review_year', 'review_month', 'review_day_of_week', 'nights']


In [ ]:
#from matplotlib import pyplot as plt
from sklearn.feature_selection import chi2 # хи-квадрат

# Предположим, что imp_cat уже заполнен
imp_cat = pd.Series(chi2(X[cat_cols], y)[0], index=cat_cols)
imp_cat.sort_values(inplace=True)
imp_cat.plot(kind='barh')


In [ ]:
from sklearn.feature_selection import f_classif # anova

imp_num = pd.Series(f_classif(X[num_cols], y)[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

In [ ]:
# Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split  

In [ ]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
# Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
# Импортируем необходимые библиотеки:  
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  
  
# Создаём модель  
regr = RandomForestRegressor(n_estimators=100)  
      
# Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)  
      
# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)  

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они отличаются  
# Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднюю абсолютную процентную ошибку предсказанных значений от фактических.  
print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))